In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 15.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633993 sha256=12549e66b0757df55a58d7e5a1204b93a27fb07a0279657e415a9c7aabcb3683
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from numba import njit
from sklearn.metrics.pairwise import cosine_similarity
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Data Glacier internship/Group_project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option("display.max_columns", None)

I downoland already prepared dataset - without NA values, outliers and duplicates. 

In [ ]:
train_df = pd.read_csv('clean_data.csv')

I will use only train file with info about products because the task is - cross selling recommendation. It is mean that client already have min 1 product. I will use item based filtration based on correlation. The clients with 0 and 1 products don't help us with analyse


In [ ]:
train_df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,First/Primary,Active,S,N,KHL,N,MALAGA,1.0,87218.100000,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,First/Primary,Inactive,S,S,KHE,N,CIUDAD REAL,0.0,35548.740000,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,First/Primary,Inactive,S,N,KHE,N,CIUDAD REAL,0.0,122179.110000,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,First/Primary,Inactive,S,N,KHD,N,ZARAGOZA,0.0,119775.540000,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,First/Primary,Active,S,N,KHE,N,ZARAGOZA,1.0,110927.085837,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


hier we have 13619537 rowas and 949612 clients. The idea is join products by clients.

In [ ]:
#change column's name

train_df.rename(columns={'ncodpers': 'customer_code', 
                         'ind_empleado': 'employee_ind',
                         'pais_residencia': 'cust_country_residence',
                         'sexo': 'sex',
                         'ind_nuevo': 'new_cust_index',
                         'antiguedad': 'customer_seniority',
                         'indresi': 'residence_ind',
                         'indext': 'foreigner_ind',
                         'conyuemp': 'spouse_index',
                         'canal_entrada': 'channel',
                         'indfall': 'deceased_ind',
                         'tipodom': 'addres_type',
                         'nomprov': 'province_name',
                         'ind_actividad_cliente': 'activity_index',
                         'renta': 'income',
                         'segmento': 'segment',
                         'ind_ahor_fin_ult1': 'Saving_Account',
                         'ind_aval_fin_ult1': 'Guarantees',
                         'ind_cco_fin_ult1': 'Current_Accounts',
                         'ind_cder_fin_ult1': 'Derivada_Account',
                         'ind_cno_fin_ult1': 'Payroll_Account',
                         'ind_ctju_fin_ult1': 'Junior_Account',
                         'ind_ctma_fin_ult1': 'Más_particular_Account',
                         'ind_ctop_fin_ult1': 'Particular_Account',
                         'ind_ctpp_fin_ult1': 'Particular_Plus_Account',
                         'ind_deco_fin_ult1': 'Short_term_deposits',
                         'ind_deme_fin_ult1': 'Medium_term_deposits',
                         'ind_dela_fin_ult1': 'Long_term_deposits',
                         'ind_ecue_fin_ult1': 'e_account',
                         'ind_fond_fin_ult1': 'Funds',
                         'ind_hip_fin_ult1': 'Mortgage',
                         'ind_plan_fin_ult1': 'Pensions',
                         'ind_pres_fin_ult1': 'Loans',
                         'ind_reca_fin_ult1': 'Taxes',
                         'ind_tjcr_fin_ult1': 'Credit_Card',
                         'ind_valo_fin_ult1': 'Securities',
                         'ind_viv_fin_ult1': 'Home_Account',
                         'ind_nomina_ult1': 'Payroll',
                         'ind_nom_pens_ult1': 'Pensions_2',
                         'ind_recibo_ult1': 'Direct_Debit'}, inplace=True)

In [ ]:
products = ['Saving_Account', 'Guarantees',
       'Current_Accounts', 'Derivada_Account', 'Payroll_Account',
       'Junior_Account', 'Más_particular_Account', 'Particular_Account',
       'Particular_Plus_Account', 'Short_term_deposits',
       'Medium_term_deposits', 'Long_term_deposits', 'e_account', 'Funds',
       'Mortgage', 'Pensions', 'Loans', 'Taxes', 'Credit_Card', 'Securities',
       'Home_Account', 'Payroll', 'Pensions_2', 'Direct_Debit']

In [ ]:
total_prod_by_cust = train_df.groupby('customer_code')[products].sum()

In [ ]:
for prod in products:
    total_prod_by_cust[prod] = total_prod_by_cust[prod].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
del_rows = total_prod_by_cust[(total_prod_by_cust[products].sum(axis=1) == 0) | (total_prod_by_cust[products].sum(axis=1) == 1)].index 

In [ ]:
total_prod_by_cust = total_prod_by_cust.drop(del_rows, axis = 0)

In [ ]:
total_prod_by_cust.head()

,Saving_Account,Guarantees,Current_Accounts,Derivada_Account,Payroll_Account,Junior_Account,Más_particular_Account,Particular_Account,Particular_Plus_Account,Short_term_deposits,Medium_term_deposits,Long_term_deposits,e_account,Funds,Mortgage,Pensions,Loans,Taxes,Credit_Card,Securities,Home_Account,Payroll,Pensions_2,Direct_Debit
customer_code,,,,,,,,,,,,,,,,,,,,,,,,
15889,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
15890,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1
15892,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1
15893,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
15894,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,1,1,1


In [ ]:
total_prod_by_cust.shape

(295654, 25)

i will use cosine similarity for recommendation

In [ ]:
df = pd.melt(total_prod_by_cust.reset_index(), id_vars=['customer_code'])
df = df.sample(frac=1, axis=0).reset_index(drop=True)
df.columns = ['customer_id', 'product_name', 'is_purchase']

In [ ]:
pivot_df = pd.pivot_table(df,index = 'customer_id',columns = 'product_name',values = 'is_purchase')
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df.fillna(0)
pivot_df = pivot_df.drop('customer_id', axis=1)

In [ ]:
pivot_df

product_name,Credit_Card,Current_Accounts,Derivada_Account,Direct_Debit,Funds,Guarantees,Home_Account,Junior_Account,Loans,Long_term_deposits,Medium_term_deposits,Mortgage,Más_particular_Account,Particular_Account,Particular_Plus_Account,Payroll,Payroll_Account,Pensions,Pensions_2,Saving_Account,Securities,Short_term_deposits,Taxes,e_account
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1
2,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,1
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295649,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
295650,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
295651,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
295652,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0


In [ ]:
co_matrix = pivot_df.T.dot(pivot_df)
np.fill_diagonal(co_matrix.values, 0)

In [ ]:
cos_score_df = pd.DataFrame(cosine_similarity(co_matrix))
cos_score_df.index = co_matrix.index
cos_score_df.columns = np.array(co_matrix.index)

In [ ]:
cos_score_df

,Credit_Card,Current_Accounts,Derivada_Account,Direct_Debit,Funds,Guarantees,Home_Account,Junior_Account,Loans,Long_term_deposits,Medium_term_deposits,Mortgage,Más_particular_Account,Particular_Account,Particular_Plus_Account,Payroll,Payroll_Account,Pensions,Pensions_2,Saving_Account,Securities,Short_term_deposits,Taxes,e_account
product_name,,,,,,,,,,,,,,,,,,,,,,,,
Credit_Card,1.000000,0.818598,0.849372,0.795940,0.867765,0.911690,0.862381,0.594361,0.839605,0.840277,0.785195,0.936436,0.784310,0.738559,0.930682,0.865322,0.851371,0.931319,0.857778,0.798596,0.907075,0.643172,0.931599,0.887339
Current_Accounts,0.818598,1.000000,0.694528,0.473216,0.742952,0.827427,0.750180,0.405574,0.586132,0.644840,0.662855,0.842122,0.510605,0.424973,0.770884,0.730669,0.742109,0.832118,0.736003,0.676300,0.763219,0.345208,0.810781,0.700333
Derivada_Account,0.849372,0.694528,1.000000,0.852986,0.961019,0.833591,0.955360,0.578993,0.930092,0.952654,0.910398,0.801474,0.848371,0.892565,0.923816,0.719431,0.730426,0.937304,0.720001,0.937516,0.939606,0.806202,0.873984,0.894768
Direct_Debit,0.795940,0.473216,0.852986,1.000000,0.857852,0.792507,0.841592,0.626113,0.874306,0.872331,0.817451,0.774415,0.827414,0.839361,0.860494,0.726652,0.705712,0.851293,0.715283,0.777427,0.874000,0.766632,0.814907,0.845737
Funds,0.867765,0.742952,0.961019,0.857852,1.000000,0.843838,0.935217,0.646653,0.889556,0.930364,0.959069,0.819390,0.876014,0.873148,0.928833,0.740098,0.752550,0.938956,0.735901,0.911575,0.953281,0.831162,0.891503,0.897503
Guarantees,0.911690,0.827427,0.833591,0.792507,0.843838,1.000000,0.827593,0.583980,0.807200,0.811960,0.706527,0.961520,0.721627,0.656928,0.907467,0.847609,0.870800,0.950478,0.853999,0.715788,0.873950,0.521437,0.912921,0.839630
Home_Account,0.862381,0.750180,0.955360,0.841592,0.935217,0.827593,1.000000,0.484315,0.936004,0.925680,0.907354,0.834042,0.778547,0.843518,0.929672,0.736573,0.754138,0.929692,0.740552,0.972119,0.956256,0.737049,0.878375,0.905648
Junior_Account,0.594361,0.405574,0.578993,0.626113,0.646653,0.583980,0.484315,1.000000,0.492511,0.721051,0.565345,0.471462,0.852058,0.552383,0.543366,0.495352,0.436649,0.580510,0.455288,0.473495,0.627481,0.699123,0.557892,0.398084
Loans,0.839605,0.586132,0.930092,0.874306,0.889556,0.807200,0.936004,0.492511,1.000000,0.922313,0.842131,0.809533,0.835071,0.934311,0.940070,0.736764,0.747321,0.887149,0.731671,0.909536,0.927744,0.801297,0.846060,0.928863


In [ ]:
product_recs = []
for i in cos_score_df.index:
    product_recs.append(cos_score_df[cos_score_df.index!=i][i].sort_values(ascending = False)[0:5].index)
    
product_recs_df = pd.DataFrame(product_recs)
product_recs_df.index = cos_score_df.index

In [ ]:
product_recs_df

,0,1,2,3,4
product_name,,,,,
Credit_Card,Mortgage,Taxes,Pensions,Particular_Plus_Account,Guarantees
Current_Accounts,Mortgage,Pensions,Guarantees,Credit_Card,Taxes
Derivada_Account,Funds,Home_Account,Long_term_deposits,Securities,Saving_Account
Direct_Debit,Loans,Securities,Long_term_deposits,Particular_Plus_Account,Funds
Funds,Derivada_Account,Medium_term_deposits,Securities,Pensions,Home_Account
Guarantees,Mortgage,Pensions,Taxes,Credit_Card,Particular_Plus_Account
Home_Account,Saving_Account,Securities,Derivada_Account,Loans,Funds
Junior_Account,Más_particular_Account,Long_term_deposits,Short_term_deposits,Funds,Securities
Loans,Particular_Plus_Account,Home_Account,Particular_Account,Derivada_Account,e_account


Hier we can see recommendation by cosine similarity